In [18]:
import os
import torch
import torch.utils.data as data
from PIL import Image
from scipy.stats import ttest_ind
from sklearn.metrics import roc_auc_score
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import transforms

In [19]:
!cp -r /kaggle/input/cable/* /kaggle/working

In [20]:
data_dir = 'cable'
train_dir = os.path.join(data_dir,'train')
test_dir = os.path.join(data_dir,'test')

In [21]:
class ImageDataset(data.Dataset):
    def __init__(self, data_dir, transform=None):
        super(ImageDataset, self).__init__()
        self.data_dir = data_dir
        self.transform = transform
        self.data = []
        self.targets = []

        # Iterate over the data directory and load images
        for root, dirs, files in os.walk(data_dir):
            for file in files:
                image_path = os.path.join(root, file)
                image = Image.open(image_path)
                self.data.append(image)

                # Determine the class label based on the directory name
                class_label = os.path.basename(root)
                if class_label == "good":
                    self.targets.append(1)  # Assign class 1 for "good"
                elif class_label == "bad":
                    self.targets.append(0)  # Assign class 0 for "bad"

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = self.data[index]
        target = self.targets[index]

        if self.transform:
            image = self.transform(image)

        return image, target

In [22]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(32 * 16 * 16, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [23]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Create data loaders
train_dataset = ImageDataset(train_dir, transform=transform)
test_dataset = ImageDataset(test_dir, transform=transform)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Set device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the CNN model
model = CNN().to(device)


# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=0.002)

In [24]:
# Training loop
num_epochs = 100
model.train()

for epoch in range(num_epochs):
    for images, targets in train_loader:
        # Move data to device
        images = images.to(device)
        targets = targets.unsqueeze(1).to(device)
        
        # Convert targets to float
        targets = targets.float()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training loss
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/100], Loss: 0.6807
Epoch [1/100], Loss: 0.2470
Epoch [1/100], Loss: 0.7146
Epoch [1/100], Loss: 0.1827
Epoch [1/100], Loss: 0.1516
Epoch [1/100], Loss: 0.2196
Epoch [1/100], Loss: 0.1956
Epoch [1/100], Loss: 0.0534
Epoch [2/100], Loss: 0.1571
Epoch [2/100], Loss: 0.4051
Epoch [2/100], Loss: 0.1751
Epoch [2/100], Loss: 0.0120
Epoch [2/100], Loss: 0.4921
Epoch [2/100], Loss: 0.1573
Epoch [2/100], Loss: 0.2095
Epoch [2/100], Loss: 0.1704
Epoch [3/100], Loss: 0.1914
Epoch [3/100], Loss: 0.3062
Epoch [3/100], Loss: 0.0409
Epoch [3/100], Loss: 0.1447
Epoch [3/100], Loss: 0.1543
Epoch [3/100], Loss: 0.5099
Epoch [3/100], Loss: 0.0103
Epoch [3/100], Loss: 0.4124
Epoch [4/100], Loss: 0.0470
Epoch [4/100], Loss: 0.1560
Epoch [4/100], Loss: 0.1684
Epoch [4/100], Loss: 0.2376
Epoch [4/100], Loss: 0.1613
Epoch [4/100], Loss: 0.3128
Epoch [4/100], Loss: 0.2302
Epoch [4/100], Loss: 0.0523
Epoch [5/100], Loss: 0.4352
Epoch [5/100], Loss: 0.1366
Epoch [5/100], Loss: 0.0412
Epoch [5/100], Loss:

In [25]:
# Evaluation
model.eval()
with torch.no_grad():
    all_predictions = []
    all_targets = []
    for images, targets in test_loader:
        
        # Move data to device
        images = images.to(device)
        targets = targets.unsqueeze(1).to(device)

        # Forward pass
        outputs = model(images)
        predictions = torch.round(outputs)

        # Collect predictions and targets
        all_predictions.extend(predictions.cpu().numpy().flatten().tolist())
        all_targets.extend(targets.cpu().numpy().flatten().tolist())

    # Calculate metrics
    correct = sum(all_predictions[i] == all_targets[i] for i in range(len(all_predictions)))
    accuracy = correct / len(all_predictions)
    tp = sum(all_predictions[i] == 1 and all_targets[i] == 1 for i in range(len(all_predictions)))
    tn = sum(all_predictions[i] == 0 and all_targets[i] == 0 for i in range(len(all_predictions)))
    fp = sum(all_predictions[i] == 1 and all_targets[i] == 0 for i in range(len(all_predictions)))
    fn = sum(all_predictions[i] == 0 and all_targets[i] == 1 for i in range(len(all_predictions)))
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    auc = roc_auc_score(all_targets, all_predictions)
    p_value = ttest_ind(all_predictions, all_targets).pvalue

In [26]:
# Print the evaluation results
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("AUC:", auc)
print("p-value:", p_value)

Accuracy: 0.4642857142857143
Sensitivity: 0.9655172413793104
Specificity: 0.10975609756097561
AUC: 0.537636669470143
p-value: 1.9111913039132994e-22
